Tutorial:
=========
Extracting a time-ordered list of processes that occurred during deposition from an AJA logfile. In order to run this, you'll have to have the required directories of AJA logfiles, recipe files, and job files.

In [20]:
#Add the AJA repository parent directory to $PATH
import sys
sys.path.append('path_to_parentdir')

import glob
import pprint

from AJA_tools import recipe_parser as aja

Specify file locations
---------------------
The easiest thing is to put all the recipe files (`.rcp`) in one folder and all the job files (`.ajp`) in another. I did this by opening a terminal and typing::

    mkdir recipes
    mv *.rcp recipes
    mkdir jobs
    mv *.ajp jobs

In [21]:
recipe_files_dir = './recipes/'
job_files_dir = './jobs/'

Parse files and generate database of jobs
---------------------------------------
This function takes in a folder of `.ajp` files and builds a dictionary of jobs that matches job names to job file locations.

In [22]:
#Build up the dictionary of jobs
jobs_dict = aja.build_jobs_dict(job_files_dir)

#Print the first couple of job names
print("The first five jobs are:")
pprint.pprint(list(jobs_dict.keys())[0:5])

The first five jobs are:
['Ci Ti deposition 3mins',
 'CLEAN_RF25W_2h_wafer3',
 'CP 30min Ti deposition_NO RF_wait_1hr',
 'CP 3X 30min Ti deposition_NO RF_wait_2hr_in_between',
 'CP Heat 4hr 250C']


Get the path to the right job file
--------------------------------
`get_job` takes in a logfile path and a dictionary of jobs (or path to a jobs folder) and returns both the job name and the job file.

In [23]:
#Specify the logfile of interest
log_file_path = './datalog/CP LU4 SiO2_200W 500nm 5rpm GUN2&GUN3 Ang 6mm RT 25RF_10262015_ 72115 PM.dlg'

job_name, job_file_path = aja.get_job(log_file_path, jobs_dict)
print("Job name is: "+job_name)
print("Job file is: "+job_file_path)
with open(job_file_path, 'r') as f:
    print(repr(f.read()))

Job name is: CP LU4 SiO2_200W 500nm 5rpm GUN2&GUN3 Ang 6mm RT 25RF
Job file is: ./jobs/CP LU4 SiO2_200W 500nm 5rpm GUN2&GUN3 Ang 6mm RT 25RF.ajp
'\x00\x00\x00\x0b\x00\x00\x00\x06Load 4\x00\x00\x00\nWait 5 min\x00\x00\x00\nWait 5 min\x00\x00\x00\x1bCP  SiO2 - STRIKE GUN2&GUN3\x00\x00\x000CP 5RPM-- Clean Presputt Gun2 and Gun3 200W 3min\x00\x00\x00)CP Dep SiO2 500nm GUN2 and GUN3 200W 25RF\x00\x00\x00"CP SiO2-- TURNOFF RF & Gun2 & Gun3\x00\x00\x00%SiO2 SL PROCESS-- TURNOFF GAS AND ROT\x00\x00\x00\nWait 5 min\x00\x00\x00\nWait 5 min\x00\x00\x00\x08Unload 4'


Extract the recipe from the job file
----------------------------------
Since the job files all have a specific format, passing in the recipe files directory path is optional. However, if that path is passed in, the function will check if each recipe step exists as a recipe file in the directory.

In [24]:
recipe_dict = aja.get_recipe(job_file_path, recipe_folder_path=recipe_files_dir)
pprint.pprint(recipe_dict)

{'info': 'raw_recipe format is: (string index, string, recipe exists?)',
 'job_name': 'CP LU4 SiO2_200W 500nm 5rpm GUN2&GUN3 Ang 6mm RT 25RF',
 'raw_job': '\x00\x00\x00\x0b'
            '\x00\x00\x00\x06Load 4\x00\x00\x00\n'
            'Wait 5 min\x00\x00\x00\n'
            'Wait 5 min\x00\x00\x00\x1bCP  SiO2 - STRIKE '
            'GUN2&GUN3\x00\x00\x000CP 5RPM-- Clean Presputt Gun2 and Gun3 200W '
            '3min\x00\x00\x00)CP Dep SiO2 500nm GUN2 and GUN3 200W '
            '25RF\x00\x00\x00"CP SiO2-- TURNOFF RF & Gun2 & '
            'Gun3\x00\x00\x00%SiO2 SL PROCESS-- TURNOFF GAS AND '
            'ROT\x00\x00\x00\n'
            'Wait 5 min\x00\x00\x00\n'
            'Wait 5 min\x00\x00\x00\x08Unload 4',
 'raw_recipe': [(0, '\x00\x00\x00\x0b\x00\x00\x00\x06', None),
                (8, 'Load 4', True),
                (14, '\x00\x00\x00\n', None),
                (18, 'Wait 5 min', True),
                (28, '\x00\x00\x00\n', None),
                (32, 'Wait 5 min', True),
  

Check for missing job files
-------------------------
Now that we've gone through all this, we may as well check to see if any of the logfiles don't have jobs associated with them.

In [25]:
logs_dict = {}

logfiles = glob.glob('./datalog/*')
for log in logfiles:
    job_name, job_file = aja.get_job(log, jobs_dict)
    if job_file is None:
        print("Bad job: "+job_name)
    else:
        logs_dict[log] = aja.get_recipe(job_file, recipe_files_dir)
        

/Users/fcarter/Documents/Code/Python/AJA_tools/recipe_parser.py:101: UserWarning: No matching job found!
  warnings.warn("No matching job found!", UserWarning)


Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: CP LU1 30min Ti deposition_NO RF_wait_1hr_5depositions_temp
Bad job: rg_lu_3_4_nb_40mins_30sec_1
Bad job: rg_lu_3_4_nb_40mins_30sec_1
Bad job: rg_lu_3_4_nb_40mins_30sec_1
Bad job: rg_lu_3_4_nb_40mins_30sec_1
Bad job: rg_lu_3_4_nb_40mins_30sec_1
Bad job: 
